In [1]:
# importanto as bibliotecas necessárias
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (ensemble, preprocessing, tree)
from sklearn.metrics import(auc, confusion_matrix, roc_auc_score,roc_curve)
from sklearn.model_selection import (train_test_split, StratifiedKFold)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from yellowbrick.classifier import ConfusionMatrix #vou chamar quando for usar. Rolou um BO no início. Pois, precisei comendar todas as bibliotecas do yellowbrick até realmente usar
from yellowbrick.classifier.rocauc import roc_auc
from yellowbrick.model_selection import LearningCurve

# Bibliotecas adicionadas durante o coding
#import pandas_profiling

In [2]:
# carregando o dataset
try:
	df = pd.read_csv('https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv')
except:
	# Alternative: use seaborn's built-in titanic dataset
	df = pd.read_csv('https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv')
orig_df = df

print(df.shape)
# pclas: classe do passageiro (1 = primeira classe, 2 = segunda classe, 3 = terceira classe)
# survived: 0 = No, 1 = Yes
# name: nome do passageiro
# sex: sexo do passageiro
# age: idade do passageiro em anos
# sibsp: número de irmãos/cônjuges a bordo
# parch: número de pais/filhos a bordo
# ticket: número do bilhete
# fare: tarifa paga pelo passageiro
# cabin: número da cabine
# embarked: porto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)
# df.boat: bote salva-vida

# organizando o dataset
dir_path='data/'
try:
	# Vou criar uma pasta para conter todas as tabelas e organizar melhor o projeto
	# exist_ok=True      Evita um erro caso o diretório já exista
    os.makedirs('data', exist_ok=True) 
    os.makedirs('data/processed', exist_ok=True)
    os.makedirs('data/preprocessed', exist_ok=True)
    # Salvar o orig_df como um arquivo csv na pasta data
    orig_df.to_csv(os.path.join(dir_path, 'orig_df.csv'), index=False) #tbm faremos o mesmo quando o df for limpo ou fracionado
except Exception as dir_path:
    print(f'Erro ao criar o diretório: {dir_path} e seus itens')

(891, 12)


# Estudo de dados para limpeza dos dados
Aqui queremos trabalho sobre os NaNs ou outliers ou dados inconsistentes. Assim como queremos ermover colunas que possam gerar algum data leakege. Tbm vamos criar colunas dummy ('pd.get_dummies')caso precisemos.
> Data leakage ocorre quando informações do conjunto de dados de teste ou validação vazam para o conjunto de treinamento durante o pré-processamento ou modelagem, ou quando informações do target vazam para as features. 

Nesse processo vamos utilizar duas bibliotecas do pandas 
- pandas_profiling
- pandas_profiling.ProfileReport(df)
Para gerar relatórios em notebooks para vermos detalhes estatísticos descritivos e dos quantis, além de histogramas

In [3]:
# exibindo as primeiras linhas do dataset
print(f'AQUI TEMOS O HEADER DE DATAFRAME IMPORTADO: \n{df.head()}\n')
print(f'COMO A MATRIZ ESTÁ \n{df.shape}\n')
print(f'OS TIPOS DE NOSSAS COLUNAS SÃO: \n{df.dtypes}\n')   # Estudamos os tipos de colunas que possuímos

# Gerar relatório de perfil (tenta ydata_profiling, senão usa pandas_profiling)
# Não importamos `display` diretamente aqui para evitar import errors com IPython.core.display
try:
    from ydata_profiling import ProfileReport
except ImportError:
    try:
        from pandas_profiling import ProfileReport
    except ImportError as e:
        raise ImportError(
            "Instale 'ydata-profiling' (recomendado) ou 'pandas-profiling' antes de gerar o relatório."
        ) from e

report = ProfileReport(df)
# Mostrar no notebook (iframe) - to_notebook_iframe() já cuida da exibição no Jupyter
try:
    report.to_notebook_iframe()
except Exception:
    # Caso a exibição inline falhe, escrevemos o arquivo HTML e informamos o caminho
    report.to_file(os.path.join(dir_path, "preprocessed", "profile_report.html"))
    print(f"Relatório salvo em {os.path.join(dir_path, 'preprocessed', 'profile_report.html')}")
else:
    # também salvar em HTML
    report.to_file(os.path.join(dir_path, "preprocessed", "profile_report.html"))

# analizando o df de forma truncada as duas colunas iniciais
df.describe().iloc[:, :2]  # Tanto `.loc` quanto `.iloc` são atributos essenciais dos DataFrames do Pandas e ambos são usados ​​para selecionar subconjuntos específicos de dados. Seu propósito é acessar e permitir a manipulação de uma parte específica do DataFrame, em vez do DataFrame inteiro.

# vamos utilizar o isnull para verificar valores nulos
print(f'O somatório de valores nulos no nosso df: {df.isnull().sum()}')
print(f'A média de valores nulos no nosso df: {df.isnull().mean()}')

# analise de algumas das linhas com dados ausentes com boleanos
mask = df.isnull().any(axis=1)
print(mask.sum(), "linhas com pelo menos um NaN")
display(df[mask].head())           # inspeciona as linhas com NaN
print(df.isnull().sum()[lambda s: s>0])  # colunas com NaNs e suas contagens
print((df.isnull().mean()*100).sort_values(ascending=False).head())  # % de NaNs por coluna
#mask.head() #linhas
#df[mask].body.head()



AQUI TEMOS O HEADER DE DATAFRAME IMPORTADO: 
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                              Heikkinen, Miss Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0  

/home/tiago-ubunto/Documentos/Estudo/ti_estudo/meus-projetos/Titanic/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|██████████| 1/1 [00:00<00:00,  1.29it/s]

Relatório salvo em data/preprocessed/profile_report.html
O somatório de valores nulos no nosso df: PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
A média de valores nulos no nosso df: PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64
708 linhas com pelo menos um NaN


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


Age         177
Cabin       687
Embarked      2
dtype: int64
Cabin          77.104377
Age            19.865320
Embarked        0.224467
PassengerId     0.000000
Name            0.000000
dtype: float64


In [ ]:
# categorização para limpeza
# defina o target antes de remover a coluna
print(f'Os tipos das colunas até então: /n{df.dtypes}/n')

print('\n') ## Tentando deixar o output mais legível
print('\n')

if 'Name' in df.columns:
    name= df['Name']
    print(f'a cabeça do nosso objeto names: \n{name.head(3)}\n')
else:
    print("A coluna do objeto Name está presente somente no dataframe original.")
#removendo colunas que não serão usadas ou podem gerar viés no modelo

print('\n')  ## Tentando deixar o output mais legível
print('\n')

df=df.drop(columns=["Name","Ticket", "Cabin"])   # Essas colunas não nos dizem muito para o modelo
# logo vamos criar uma coluna "dummy"
df=pd.get_dummies(df)  # ou # df=pd.get_dummies(df,drop_first=True) #para evitar a dummy trap... No caso as colunas male_sex e famale são inversos perfeitos
df=df.drop(columns=["Sex_female"])  # removendo uma das colunas dummies para evitar dummy trap
print(f'Nossas colunas são com os dummies aplicados: {df.columns}') 

y = df['Survived']   #Estou em dúvida se criaria outra coluna "dead" como target alternativo. Se pensarmos em Boleanos, seriam antônimos
X = df.drop(columns="Survived")

print('\n')
print('\n')

print(f'Nossa matriz de features é: \n{X.head()}\n')

#pclass: classe do passageiro (1 = primeira classe, 2 = segunda classe, 3 = terceira classe)
#name
#sex
#age
#sibsp: número de irmãos/cônjuges a bordo
#parch: número de pais/filhos a bordo
#ticket: número do bilhete
#fare: tarifa paga pelo passageiro
#cabin: número da cabine
#embarked: porto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)

Os tipos das colunas até então: /nPassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object/n




a cabeça do nosso objeto names: 
0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                Heikkinen, Miss Laina
Name: Name, dtype: object





Nossas colunas são com os dummies aplicados: Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [5]:
# Separação de amostra #amostra_para_validação_do_modelo
# scikit-learn para separar 30% para os testes
# import library q usaremos abaixo
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

#Impotação dos dados
##como a coluna idade tem valores ausentes. Vamos imputar udade a partir dos valores numéricos apenas no conjunto de treinamento, e então

In [6]:
# IterativeImputer é experimental em algumas versões do sklearn:
# importe o "enable" antes de importar o estimador
from sklearn.experimental import enable_iterative_imputer  # deve vir antes DO DE BAIXO
from sklearn.impute import IterativeImputer

# Colunas numéricas a imputar (use nomes como strings)
num_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

# verificar quais colunas realmente existem nos dataframes (evita KeyError)
num_cols = [c for c in num_cols if c in X_train.columns]

imputer = IterativeImputer(random_state=0)
X_train.loc[:, num_cols] = imputer.fit_transform(X_train[num_cols])
X_test.loc[:, num_cols] = imputer.transform(X_test[num_cols])

# preencher possíveis NaNs restantes com a mediana do conjunto de treino (aplicado às colunas numéricas)
meds = X_train[num_cols].median()
X_train.loc[:, num_cols] = X_train[num_cols].fillna(meds)
X_test.loc[:, num_cols] = X_test[num_cols].fillna(meds)

In [7]:
#normalização dos dados
## Este processo será importante para nosso modelo ter uma melhor performance.
cols = ["Pclass", "Age", "Sibsp", "Fare"]
num_cols = [c for c in cols if c in X_train.columns]
sca = preprocessing.StandardScaler()

# Escalar apenas as colunas numéricas selecionadas no lugar, preservando as demais colunas (dummies)
if num_cols:
    X_train.loc[:, num_cols] = sca.fit_transform(X_train[num_cols])
    X_test.loc[:, num_cols] = sca.transform(X_test[num_cols])


# Conferir tipos e shapeprint("Numeric cols scaled:", num_cols)
print("X_train.shape:", X_train.shape)

X_train.shape: (623, 10)


/tmp/ipykernel_82433/1525818701.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.63788124  0.80326712  0.80326712 -0.41730706 -0.41730706 -1.63788124
 -0.41730706  0.80326712  0.80326712 -0.41730706 -1.63788124 -0.41730706
  0.80326712  0.80326712 -1.63788124  0.80326712 -0.41730706  0.80326712
  0.80326712  0.80326712 -0.41730706  0.80326712  0.80326712  0.80326712
  0.80326712  0.80326712  0.80326712 -1.63788124  0.80326712 -0.41730706
  0.80326712  0.80326712  0.80326712 -1.63788124 -0.41730706 -0.41730706
  0.80326712 -1.63788124  0.80326712  0.80326712 -1.63788124 -0.41730706
 -1.63788124 -0.41730706 -0.41730706  0.80326712  0.80326712 -0.41730706
  0.80326712 -0.41730706 -0.41730706  0.80326712 -1.63788124  0.80326712
 -1.63788124  0.80326712 -0.41730706  0.80326712  0.80326712  0.80326712
  0.80326712  0.80326712  0.80326712  0.80326712  0.80326712 -0.41730706
 -0.41730706 -1.63788124 -1.63788124 -0.4

In [8]:
#Refatoração
## A medida que o ML for evoluindo, vamos precisar realizar ajustes e até mesmo importar o original dataset novamente para refinar o modelo.
## Faremos duas funções para o processo

def tweak_titanic(df):
    """Remove colunas que causam leakage e converte categóricas em dummies (drop_first=True).
    Retorna um DataFrame somente com colunas numéricas e dummies.
    """
    df = df.copy()
    drop_cols = [c for c in ["Name", "Ticket", "Survived", "Cabin"] if c in df.columns]
    if drop_cols:
        df = df.drop(columns=drop_cols)
    df = pd.get_dummies(df, drop_first=True)
    return df


def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
    """Separa X/y, divide treino/teste, imputa colunas numéricas e (opcional) escala colunas em std_cols.
    Retorna X_train, X_test, y_train, y_test.
    """
    if y_col not in df.columns:
        raise ValueError(f"{y_col} não encontrado em df.columns")

    y = df[y_col]
    X = df.drop(columns=y_col)

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=size, stratify=y, random_state=42
    )

    # Colunas numéricas candidatas (ajuste conforme seu df)
    num_candidates = ["Pclass", "Age", "SibSp", "Sibsp", "Parch", "Fare"]
    num_cols = [c for c in num_candidates if c in X_train.columns]

    if num_cols:
        from sklearn.experimental import enable_iterative_imputer  # noqa: F401
        from sklearn.impute import IterativeImputer

        imputer = IterativeImputer(random_state=0)
        X_train.loc[:, num_cols] = imputer.fit_transform(X_train[num_cols])
        X_test.loc[:, num_cols] = imputer.transform(X_test[num_cols])

    # Normalizar colunas passadas em std_cols (aceita string "a,b,c" ou lista)
    if std_cols:
        if isinstance(std_cols, str):
            std_cols = [c.strip() for c in std_cols.split(",")]
        std_cols = [c for c in std_cols if c in X_train.columns]
        if std_cols:
            std = preprocessing.StandardScaler()
            X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
            X_test.loc[:, std_cols] = std.transform(X_test[std_cols])

    return X_train, X_test, y_train, y_test
    # Escalar colunas passadas em std_cols (somente se existirem)

In [9]:
#modelo_de_base
#uma_base_realmente_simples_que_o_modelo_usara_como_comparador

from sklearn.dummy import DummyClassifier
bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test) #nos_dará_nossa_precisao

from sklearn import metrics
metrics.precision_score(y_test, bm.predict(X_test))



#familias
#Neste_projeto_vamos_comparar_pontuações_AUC_e_o_desvio_padão_usando_a_validação_cruzadak_fold
X=pd.concat([X_train, X_test])
y=pd.concat([y_train, y_test])

from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

models = [
    DummyClassifier(),
    LogisticRegression(max_iter=1000),
    DecisionTreeClassifier(random_state=42),
    KNeighborsClassifier(),
    GaussianNB(),
    SVC(probability=True),
    RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42),

    XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),

]

## Cross-validation stratificada com 10 folds
cv = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)    

for m in models:
    try:
        s = cross_val_score(m, X, y, scoring="roc_auc", cv=cv, n_jobs=-1, error_score='raise')
        print(f'{m.__class__.__name__:22} AUC: {s.mean():.3f} STD: {s.std():.2f}')  #Por que usar o AUC em vez da Acurácia? A acurácia pode ser enganosa se os seus dados estiverem desbalanceados.
    except Exception as e:
        print(f'{m.__class__.__name__:22} FAILED!!!!!: {e}')

/home/tiago-ubunto/Documentos/Estudo/ti_estudo/meus-projetos/Titanic/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


DummyClassifier        AUC: 0.500 STD: 0.00
LogisticRegression     AUC: 0.852 STD: 0.02
DecisionTreeClassifier AUC: 0.738 STD: 0.03
KNeighborsClassifier   AUC: 0.541 STD: 0.06
GaussianNB             AUC: 0.818 STD: 0.02
SVC                    AUC: 0.788 STD: 0.03
RandomForestClassifier AUC: 0.872 STD: 0.01
XGBClassifier          AUC: 0.848 STD: 0.03


/home/tiago-ubunto/Documentos/Estudo/ti_estudo/meus-projetos/Titanic/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [15:50:08] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/tiago-ubunto/Documentos/Estudo/ti_estudo/meus-projetos/Titanic/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [15:50:08] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/tiago-ubunto/Documentos/Estudo/ti_estudo/meus-projetos/Titanic/.venv/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [15:50:08] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/tiago-ubunto/Documentos/Estudo/ti_estudo/meus-projetos/Titanic/.venv/lib/python3.12/site-packages/xgboost/training.py:199: User

In [10]:
# Criando o modulo
## Usaremos um classificador de random forest
rf=ensemble.RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [11]:
#Avaliação de modelo
rf.score(X_test, y_test)
metrics.precision_score(y_test, rf.predict(X_test))

for col, val in sorted(
zip(X_train.columns, rf.feature_importances_,), key=lambda x: x[1], reverse=True,)[:5]):
    print(f'{col:10}{val:10.3f}')

SyntaxError: unmatched ')' (1172146597.py, line 6)

In [ ]:
#Otimizando o modelo
